# Тестовое задание

В данном задании необходимо написать код, который считает следующие признаки для каждого клиента `user_id`:

1. `events_ordinal_number` - порядковый номер события
2. `second_event_time` - время второго события
2.	`loan_ordinal_number` - порядковый номер займа
2.	`events_time_diff` - разница во времени между событиями
2.	`previous_loans_max_amount` - максимальная сумма предыдущего займа.


### Подготовка датафрейма

In [164]:
import numpy as np
import pandas as pd
from datetime import datetime

In [165]:
size = 10000

df = pd.DataFrame(
    {
        'time': [*pd.date_range('2001-01-01', '2020-09-01', freq='1h')][:size],
        'user_id': np.random.choice(1000, size),
        'type': np.random.choice(2, size, p=[0.8, 0.2]),
        'amount': [int(x//1) for x in  np.random.normal(15_000, 1_000, size)],
        'target': np.random.choice(2, size, p=[0.5, 0.5]),
    },
    index=[*range(size)],
)

In [166]:
df['is_loan'] = df['type']
df['type'] = df['type'].replace({0: 'loanRequest', 1: 'loan'})

In [167]:
df.to_csv('test_origin.csv', index=False)

### Расчет признаков

#### Признак `events_ordinal_number`

In [168]:
# весь код расчета признака должен быть в этом методе
def calculate_events_number(df):
    df['events_ordinal_number'] = df.sort_values('time').groupby('user_id').cumcount()+1
    return df


In [169]:
calculate_events_number(df).sort_values(by=['user_id','time']).head(30)

,time,user_id,type,amount,target,is_loan,events_ordinal_number
913,2001-02-08 01:00:00,0,loanRequest,15192,0,0,1
1109,2001-02-16 05:00:00,0,loanRequest,13831,1,0,2
1523,2001-03-05 11:00:00,0,loanRequest,15580,1,0,3
1967,2001-03-23 23:00:00,0,loanRequest,16419,0,0,4
2085,2001-03-28 21:00:00,0,loanRequest,16067,0,0,5
2787,2001-04-27 03:00:00,0,loanRequest,16969,1,0,6
3208,2001-05-14 16:00:00,0,loanRequest,14689,1,0,7
3392,2001-05-22 08:00:00,0,loan,14605,1,1,8
4335,2001-06-30 15:00:00,0,loanRequest,13547,1,0,9
4754,2001-07-18 02:00:00,0,loanRequest,14525,0,0,10


#### Признак `second_event_time`

In [170]:
# весь код расчета признака должен быть в этом методе
def calculate_second_event_time(df):
    df = calculate_events_number(df)
    second_event = df[df['events_ordinal_number']== 2].groupby('user_id')[['user_id','time']].apply(lambda x:x)
    second_event.rename(columns={'time':'second_event_time'},inplace=True)
    df = df.merge(second_event,on='user_id')
    return df

In [171]:
#calculate_second_event_time(df).sort_values(by=['user_id','time']).head(30)

#### Признак `loan_ordinal_number`

In [172]:
# весь код расчета признака должен быть в этом методе
def calculate_loan_number(df):
    #df = calculate_second_event_time(df)
    df['loan_ordinal_number'] = df.sort_values('time').groupby('user_id').is_loan.apply(lambda x: x.shift(1).fillna(1).cumsum())
    return df

In [173]:
#calculate_loan_number(df).sort_values(by=['user_id','time']).head(30)

#### Признак `events_time_diff`

In [174]:
# весь код расчета признака должен быть в этом методе
def calculate_time_diff(df):
    #df = calculate_loan_number(df)
    df['events_time_diff'] = df.sort_values(by=['time']).groupby('user_id').time.apply(lambda x: abs(x.shift(1) - x))
    return df

In [175]:
#calculate_time_diff(df).sort_values(by=['user_id','time']).head(30)

#### Признак `previous_loans_max_amount`

In [176]:
# весь код расчета признака должен быть в этом методе
def calculate_previous_max_amount(df):
    #df = calculate_time_diff(df)
    df['loan'] = df['amount'] * df['is_loan'].replace({0: np.nan})
    df['previous_loans_max_amount'] = df.sort_values(by=['time']).groupby('user_id').loan.apply(lambda x: x.shift(1).ffill())
    df.drop(['loan'], axis=1, inplace=True)
    return df

In [177]:
#calculate_previous_max_amount(df).sort_values(by=['user_id','time']).head(30)

### Сохранение итогового датасета с признаками

Этот код трогать не нужно.

In [178]:
def calculate_feature(df):
    calculate_events_number(df)
    calculate_second_event_time(df)
    calculate_loan_number(df)
    calculate_time_diff(df)
    calculate_previous_max_amount(df)

In [179]:
%%timeit -o 
calculate_feature(df)

4.64 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 4.64 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [180]:
columns = ['time']
rez = _
df_time = pd.DataFrame([rez], None, columns)
df_time.to_csv('time.csv', index=False)

In [181]:
df.to_csv('test.csv', index=False)